In [1]:
from langchain import hub
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.supabase import SupabaseVectorStore
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from typing import Dict, Any, Tuple, List
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.tools.tavily_search import TavilySearchResults

In [2]:
from utils.utils import initialize_environment_variables, initialize_subabase_client, initialize_openai_client
from utils.utils import initialize_tavily_client

initialize_environment_variables("../.env")

# Initialize Supabase Client
supabase_client = initialize_subabase_client()

# Initialize OpenAI Client
openai_client = initialize_openai_client()

# Initialize Tavily Client
tavily_client = initialize_tavily_client()

In [3]:
input_dict = {
"school_type": "Gymnasium",
"subject": "English",
"topic": "Shakespeare",
"grade": "13.",
"state": "Bayern",
"keywords": ["shakespeare", "poetry", "analysis"], 
"context": ["Pedagogy.pdf"]
}

In [4]:
def unpack_prompt_input(input: Dict[str, Any]) -> Tuple[str, str, str, str, List[str], List[str]]:
    school_type = str(input.get("subject"))
    subject = str(input.get("subject"))
    topic = str(input.get("topic"))
    grade = str(input.get("grade"))
    state = str(input.get("state"))
    keywords = [str(i) for i in input.get("keywords")]
    context = [str(i) for i in input.get("context")]
    return school_type, subject, topic, grade, state, keywords, context

In [5]:
school_type, subject, topic, grade, state, keywords, context = unpack_prompt_input(input_dict)

In [6]:
task = f"Plane eine Unterrichtsstunde im Fach {subject} zum Thema {topic} für eine {grade} Klasse in {state} für den Schultyp {school_type}."

In [11]:
filter_dict = {"source": [str("docs/" + i) for i in context]}

In [21]:
filter_dict = {'source':'docs/Pedagogy.pdf', "source":'docs/LIS_PDF_29-01-2024.pdf'}

In [22]:
embeddings = OpenAIEmbeddings()
vector_store = SupabaseVectorStore(
    client=supabase_client,
    embedding=embeddings,
    table_name="documents_new",
)
relevant_chunks = vector_store.similarity_search(task,k=2,filter=filter_dict)

2024-01-29 20:47:50,066:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-01-29 20:47:50,212:INFO - HTTP Request: POST https://smxwnqdpxcvclxhbvhsf.supabase.co/rest/v1/rpc/match_documents?limit=2 "HTTP/1.1 200 OK"


In [23]:
relevant_chunks

[Document(page_content='Lehrplanauszüge\n29.01.2024 1Fachlehrpläne\nGymnasium: Englisch 12/13 (grundlegendes  \nAnforderungsniveau)\ngültig ab Schuljahr 2024/25\nE12/13 1: Kommunikative Kompetenzen\nE12/13 1.1: Kommunikative Fertigkeiten\nE12/13 : Hör- und Hörsehverstehen\nKompetenzerwartungen und Inhalte\nDie Schülerinnen und Schüler ...\n•entnehmen einem authentischen Hör- bzw. Hörsehtext die  \nHauptaussagen oder Einzelinformationen entsprechend der Hör-  \nbzw. Hörseh-Absicht, auch wenn Hintergrundgeräusche oder die Art  \nder Wiedergabe das Verstehen beeinﬂussen.\n•kombinieren textinterne Informationen und breites textexternes  \nWissen und erfassen ggf. die Wirkungsabsicht des Textes.\n•wenden in Abhängigkeit von der jeweiligen Hör-/Hörseh-Absicht  \nRezeptionsstrategien an, auch unter Berücksichtigung spezifischer  \naudiovisueller Gestaltungsmittel.\n•setzen angemessene Strategien zur Lösung von  \nVerständnisproblemen auch bei unbekanntem, nicht immer  \nerschließbarem Sprachm

In [9]:
prompt_template = f"""Du bist ein Assistent für Lehrende und hilfst ihnen dabei ihren Unterricht vorzubereiten. 
Verwende dazu die Informationen im Kontext, um die Aufgabe am Ende zu lösen. Gib den Schlüsselwötern bei der Lösung der Aufgabe ein besonders hohes Gewicht.
Wenn du keine Lösung findest, sage einfach, dass du die Aufgabe nicht lösen kannst, und versuche keine Antwort zu erfinden. 
Die Lösung der Aufgabe soll eine Struktur der angefragten Unterrichtsstunde enthalten inklusive einer zeitlichen Struktur.

Schlüsselwörter:
---------
{keywords}
---------

Kontext:
---------
{context}
---------

Los geht's!

Aufgabe: {task}"""

In [10]:
prompt_template

"Du bist ein Assistent für Lehrende und hilfst ihnen dabei ihren Unterricht vorzubereiten. \nVerwende dazu die Informationen im Kontext, um die Aufgabe am Ende zu lösen. Gib den Schlüsselwötern bei der Lösung der Aufgabe ein besonders hohes Gewicht.\nWenn du keine Lösung findest, sage einfach, dass du die Aufgabe nicht lösen kannst, und versuche keine Antwort zu erfinden. \nDie Lösung der Aufgabe soll eine Struktur der angefragten Unterrichtsstunde enthalten inklusive einer zeitlichen Struktur.\n\nSchlüsselwörter:\n---------\n['shakespeare', 'poetry', 'analysis']\n---------\n\nKontext:\n---------\n['pdfId#1', 'pdfId#2']\n---------\n\nLos geht's!\n\nAufgabe: Plane eine Unterrichtsstunde im Fach English zum Thema Shakespeare für eine 13. Klasse in Bayern ."

In [8]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [9]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

In [10]:
embeddings = OpenAIEmbeddings()
vector_store = SupabaseVectorStore(
        client=supabase_client,
        embedding=embeddings,
        table_name="documents_new",
        query_name="match_documents_mmr",
    )
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 3, 'lambda_mult': 0.25})
retriever_tool = tool = create_retriever_tool(
    retriever,
    "search_context",
    "Search relevant context information.",
)

In [1]:
tools = [retriever_tool, tavily_tool]

NameError: name 'retriever_tool' is not defined

In [2]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
agent = create_openai_functions_agent(llm, tools, prompt)

NameError: name 'ChatOpenAI' is not defined

In [13]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [14]:
agent_executor.invoke({"input": prompt_template})



> Entering new AgentExecutor chain...


2024-01-29 12:10:07,709:INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Hier ist ein Vorschlag für die Struktur einer Unterrichtsstunde zum Thema Integrale:

1. **Einführung (10 Minuten):** Beginnen Sie mit einem kurzen Rückblick auf die vorherigen Lektionen, die sich mit der Differentialrechnung beschäftigt haben. Erklären Sie dann, dass der heutige Unterricht sich auf das Thema Integrale konzentriert, welches das Gegenstück zur Differentialrechnung ist.

2. **Theoretischer Teil (20 Minuten):** Führen Sie die Schüler in das Konzept der Integralrechnung ein. Erklären Sie, was ein Integral ist, und zeigen Sie, wie es graphisch dargestellt wird. Stellen Sie die Formel für das unbestimmte Integral vor und gehen Sie durch einige einfache Beispiele, um zu zeigen, wie es berechnet wird.

3. **Praktischer Teil (20 Minuten):** Lassen Sie die Schüler einige Übungsaufgaben zur Berechnung von Integralen machen. Beginnen Sie mit einfachen Beispielen und steigern Sie dann allmählich den Schwierigkeitsgrad. Gehen Sie im Klassenzimmer herum und helfen Sie den Schülern be

{'input': "Du bist ein Assistent für Lehrerende und hilfst ihnen dabei ihren Unterricht vorzubereiten. Verwende relevante Kontextinformationen, um die Aufgabe am Ende zu lösen. Wenn du keine Lösung findest, sage einfach, dass du die Aufgabe nicht lösen kannst, und versuche keine Antwort zu erfinden. Die Lösung der Aufgabe soll eine Struktur der angefragten Unterrichtsstunde enthalten zusammen mit groben Zeitvorschlägen.\n\nLos geht's!\n\nAufgabe: Plane eine Unterrichtsstunde im Fach Mathematik zum Thema Integrale für eine 13. Klasse in Bayern.",
 'output': 'Hier ist ein Vorschlag für die Struktur einer Unterrichtsstunde zum Thema Integrale:\n\n1. **Einführung (10 Minuten):** Beginnen Sie mit einem kurzen Rückblick auf die vorherigen Lektionen, die sich mit der Differentialrechnung beschäftigt haben. Erklären Sie dann, dass der heutige Unterricht sich auf das Thema Integrale konzentriert, welches das Gegenstück zur Differentialrechnung ist.\n\n2. **Theoretischer Teil (20 Minuten):** Füh

In [6]:
embeddings = OpenAIEmbeddings()

In [32]:
vector_store = SupabaseVectorStore(
        client=supabase_client,
        embedding=embeddings,
        table_name="documents_new",
        query_name="match_documents_mmr",
    )

In [64]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 3, 'lambda_mult': 0.25})

In [65]:
mmr_docs = retriever.get_relevant_documents(query)

2024-01-28 22:24:14,841:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-01-28 22:24:15,070:INFO - HTTP Request: POST https://smxwnqdpxcvclxhbvhsf.supabase.co/rest/v1/rpc/match_documents_mmr?limit=20 "HTTP/1.1 200 OK"


In [66]:
mmr_docs

[Document(page_content='11. Chazan, Barry (2022). "What is "Education"?" (https://link.springer.com/chapter/10.1007/978-\n3-030-83925-3_3). Principles and Pedagogies in Jewish Education. Springer International\nPublishing. pp. 13–21. doi:10.1007/978-3-030-83925-3_3 (https://doi.org/10.1007%2F978-3-\n030-83925-3_3). ISBN 978-3-030-83925-3. S2CID 239896844 (https://api.semanticscholar.o\nrg/CorpusID:239896844). Archived (https://web.archive.org/web/20220512120351/https://lin\nk.springer.com/chapter/10.1007/978-3-030-83925-3_3) from the original on 12 May 2022.\nRetrieved 13 May 2022.\n12. Marshall, James D. (2006). "The Meaning of the Concept of Education: Searching for the\nLost Arc" (https://www.jstor.org/stable/42589880). Journal of Thought. 41 (3): 33–37.\nISSN 0022-5231 (https://www.worldcat.org/issn/0022-5231). JSTOR 42589880 (https://www.\njstor.org/stable/42589880). Archived (https://web.archive.org/web/20220512120351/https://w\nww.jstor.org/stable/42589880) from the original on 

In [44]:
matched_docs = vector_store.similarity_search_with_relevance_scores(query)

2024-01-28 22:02:48,795:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-01-28 22:02:48,978:INFO - HTTP Request: POST https://smxwnqdpxcvclxhbvhsf.supabase.co/rest/v1/rpc/match_documents_mmr?limit=4 "HTTP/1.1 200 OK"


In [45]:
matched_docs

[(Document(page_content='41. UNESCO (2019). Empowering students for just societies: a handbook for secondary school\nteachers (https://unesdoc.unesco.org/ark:/48223/pf0000370901?posInSet=2&queryId=a055\n95b4-f5b2-48ce-b649-f8177b6036fe). UNESCO. ISBN 978-92-3-100340-0.\n42. Kincheloe, Joe (2008). Critical Pedagogy Primer. New York: Peter Lang.\nISBN 9781433101823.\n43. Kincheloe, Joe L.; Horn, Raymond A., eds. (2007). The Praeger Handbook of Education and\nPsychology (https://books.google.com/books?isbn=0313331235). Praeger. p. 552.\nISBN 978-0313331237.\n44. Jones, Leo. (2007). The Student-Centered Classroom. Cambridge University Press.\n45. Rogers, C. R. (1983). Freedom to Learn for the \'80s. New York: Charles E. Merrill Publishing\nCompany, A Bell & Howell Company.\n46. Pedersen, S., & Liu, M. (2003). Teachers\' beliefs about issues in the implementation of a\nstudent-centered learning environment. Educational Technology Research and\nDevelopment, 51(2), 57–76.\n47. Hannafin, M. J.

In [53]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [54]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [55]:
from langchain_openai import ChatOpenAI

In [67]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [68]:
for chunk in rag_chain.stream(query):
    print(chunk, end="", flush=True)

2024-01-28 22:24:51,115:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-01-28 22:24:51,314:INFO - HTTP Request: POST https://smxwnqdpxcvclxhbvhsf.supabase.co/rest/v1/rpc/match_documents_mmr?limit=20 "HTTP/1.1 200 OK"
2024-01-28 22:24:52,331:INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Education is the theory and practice of learning, encompassing the imparting of knowledge and skills in an educational context. It involves the interactions between teachers and students and is influenced by social, political, and psychological factors. Pedagogy can be seen as both a science and an art, with various approaches and methods used to achieve educational goals.

In [11]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="map_rerank", return_intermediate_steps=True, prompt=PROMPT)

In [12]:
query_embeddings = embeddings.embed_query(query)

2024-01-28 21:03:41,017:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [13]:
relevant_chunks = vector_store.similarity_search_by_vector_with_relevance_scores(query_embeddings,k=2)

2024-01-28 21:03:50,734:INFO - HTTP Request: POST https://smxwnqdpxcvclxhbvhsf.supabase.co/rest/v1/rpc/match_documents?limit=2 "HTTP/1.1 200 OK"


In [11]:
chunk_docs=[]
for chunk in relevant_chunks:
        chunk_docs.append(chunk[0])

In [12]:
results = chain({"input_documents": chunk_docs, "question": query})

/home/cts/.cache/pypoetry/virtualenvs/unterricht-python-backend-9Hag_kgR-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/cts/.cache/pypoetry/virtualenvs/unterricht-python-backend-9Hag_kgR-py3.10/lib/python3.10/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
2024-01-27 18:04:01,009:INFO - HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"


In [13]:
text_reference=""
for i in range(len(results["input_documents"])):
    text_reference+=results["input_documents"][i].page_content
output={"Answer":results["output_text"],"Reference":text_reference}

In [14]:
output

{'Answer': ' Unterricht is a German word that translates to "instruction" or "teaching." It refers to the process of imparting knowledge or skills to students in a formal setting, such as a classroom or online course. ',
 'Reference': 'Paulo Reglus Neves Freire (September 19, 1921 – May 2, 1997) was a Brazilian educator and\nphilosophe r who was a leading advoc ate of critical pedagogy . He is best know n for his influential work\nPedagogy  of the Oppressed, which is generally considered one of the founda tional texts of the critical\npedagogy m ovement.[26][27][28]\nConfucius (551–479 BCE) stated that authority has the respons ibility to provide oral and written instruction\nto the people unde r the rule, and "should do them good in every possible way."[21] One of the deepest\nteachings of Confucius may have been the superiority of personal exemplification over explicit rules of\nbehavior. His moral teachings emphasized self-cultivation, emulation of moral exemplars, and the\nattainme

In [15]:
def getanswer(query):
    embeddings = OpenAIEmbeddings()
    vector_store = SupabaseVectorStore(
        client=supabase_client,
        embedding=embeddings,
        table_name="documents_new",
    )
    query_embeddings = embeddings.embed_query(query)
    relevant_chunks = vector_store.similarity_search_by_vector_with_relevance_scores(query_embeddings,k=2)
    chunk_docs=[]
    for chunk in relevant_chunks:
        chunk_docs.append(chunk[0])
    results = chain({"input_documents": chunk_docs, "question": query})
    text_reference=""
    for i in range(len(results["input_documents"])):
        text_reference+=results["input_documents"][i].page_content
    output={"Answer":results["output_text"],"Reference":text_reference}
    return output

In [16]:
getanswer("Was ist Unterricht?")

2024-01-27 18:04:10,677:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-01-27 18:04:10,856:INFO - HTTP Request: POST https://smxwnqdpxcvclxhbvhsf.supabase.co/rest/v1/rpc/match_documents?limit=2 "HTTP/1.1 200 OK"
/home/cts/.cache/pypoetry/virtualenvs/unterricht-python-backend-9Hag_kgR-py3.10/lib/python3.10/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
2024-01-27 18:04:12,146:INFO - HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"


{'Answer': ' Unterricht ist ein Prozess, bei dem Wissen und Fähigkeiten vermittelt werden, um das Verständnis und die Fähigkeiten der Lernenden zu fördern. Es kann in verschiedenen Formen stattfinden, wie z.B. im Klassenzimmer, online oder durch Fernunterricht.',
 'Reference': 'Paulo Reglus Neves Freire (September 19, 1921 – May 2, 1997) was a Brazilian educator and\nphilosophe r who was a leading advoc ate of critical pedagogy . He is best know n for his influential work\nPedagogy  of the Oppressed, which is generally considered one of the founda tional texts of the critical\npedagogy m ovement.[26][27][28]\nConfucius (551–479 BCE) stated that authority has the respons ibility to provide oral and written instruction\nto the people unde r the rule, and "should do them good in every possible way."[21] One of the deepest\nteachings of Confucius may have been the superiority of personal exemplification over explicit rules of\nbehavior. His moral teachings emphasized self-cultivation, emul